## Deep Q Learning on Cart-pole

### MD Muhaimin Rahman
contact: sezan92[at]gmail[dot]com

In this project, I have worked on Deep Q Learning on Cart-Pole. The concept is based on the ground baking [work](https://arxiv.org/pdf/1312.5602.pdf) of Deep Mind, which they tested on Atari Games. I am showing here simple implementation on simpler game. The CartPole video is available [here](https://youtu.be/IKobsp1WszI)

![CartPole](Cartpole.png)

Importing Packages

In [8]:
import gym
import numpy as np
from collections import deque
import random
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

Setting up Environment

In [2]:
env = gym.make('CartPole-v0')

[2018-03-06 22:30:47,690] Making new env: CartPole-v0


Hiperparameters

In [3]:
legal_actions=env.action_space.n
actions = [0,1]
gamma =0.95
lr =0.5
num_episodes =1000
epsilon =1
epsilon_decay =0.995
memory_size =1000
batch_size=100
show=False

Change show into ```True``` for visualization 

### The Concept of Experience Replay

One of the challenges of Deep Learning based Q Learning algorithms is that, there is a high chance that every state of an agent is related to previous states. Which may not be the case always. Specially when we are giving actions randomly. So we need random states with random actions.


Here comes Experience Replay, which is nothing. Let the agent take random actions for , say 1000 times. Save those data into memory. While training the agent, feed the data randomly to the architecture. Also update the memory with new data while training goes on.

So our steps are as following
* Initialize Memory
* Define Deep Learning Architecture
* Save N number of Random states with their rewards , actions and resulting states
* Start Episode
* Take some actions based on $\epsilon$ greedy policy
* Save those states,actions, rewards and new states in the memory
* Take random minibatch from the memory and train the architecture based on the minibatch
* Start New Episode


For our memory, we will use special data structure of python ```deque``` . It is a special type of list which has a limit of its elements. The moment you append another element it will remove the first element of it. It is working like a open tube with two open ends

In [4]:
memory=deque(maxlen=memory_size)
s=env.reset()
s = s.reshape((1,-1))
a=env.action_space.sample()
new_s,r,d,_ =env.step(a)
new_s = new_s.reshape((1,-1))
experience=(s,r,a,new_s)
memory.append(experience)
s = new_s
for _ in range(memory_size):
    a=env.action_space.sample()
    new_s,r,d,_ =env.step(a)
    new_s = new_s.reshape((1,-1))
    if show:
        env.render()
    if d:
        r=-100
        experience =(s,r,a,new_s)
        s=env.reset()
        s = s.reshape((1,-1))
    else:    
        experience =(s,r,a,new_s)
    memory.append(experience)
    s = new_s
env.close() 

### Deep Q Learning

Vanilla Q learning works best where the state and actions of the agent is discrete . Like the following diagram , which is taken from [A painless Q Learning Tutorial](http://mnemstudio.org/path-finding-q-learning-tutorial.htm) 

![Painless](q_matrix5.gif)

Here the actions[along rows] are discrete as 0-5 and states [along columns] are also 0-5 . For state 0 the maximum Q value is 80 which is in column 4. So the best action will be 4. But in our case, the Cartpole game, the states aren't discrete. We had to discretize in the previous [version](https://github.com/sezan92/CartPoleRL/blob/master/CartPoleQLearning.ipynb).

Other than discretization , we cannot make the infinite states. So the best way is to train a neural network ,which given a state , gives output of Q value for each action. The agent will choose the action with maximum Q value and it will go on.

![DQL](DQL.png)

Our Model

In [5]:
model = Sequential()
model.add(Dense(20,activation='relu',input_shape=(1,4)))
model.add(Dense(20,activation='relu'))
model.add(Dense(2,activation='linear'))
model.compile(loss='mse',optimizer=Adam(lr=0.01),)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1, 20)             100       
_________________________________________________________________
dense_2 (Dense)              (None, 1, 20)             420       
_________________________________________________________________
dense_3 (Dense)              (None, 1, 2)              42        
Total params: 562
Trainable params: 562
Non-trainable params: 0
_________________________________________________________________


Training

In [6]:
ep_list =[]
reward_list =[]  
for ep in range(num_episodes):
    s= env.reset()
    s=s.reshape((1,-1))
    rAll =0
    d = False
    j = 0
    for j in range(200):
        #time.sleep(0.01)
        #epsilon greedy. to choose random actions initially when Q is all zeros
        if np.random.random()< epsilon:
            a = np.random.randint(0,legal_actions)
            #epsilon = epsilon*epsilon_decay
        else:
            Q = model.predict(s.reshape(-1,s.shape[0],s.shape[1]))
            a =np.argmax(Q)
        new_s,r,d,_ = env.step(a)
        new_s = new_s.reshape((1,-1))
        rAll=rAll+r
        if show:
            env.render()
        if d:
            if rAll<195:
                r =-100
                experience = (s,r,a,new_s)
                memory.append(experience)
                print("Episode %d, Failed! Reward %d"%(ep,rAll))
                #break
            elif rAll>195:
                print("Episode %d, Passed! Reward %d"%(ep,rAll))
            ep_list.append(ep)
            reward_list.append(rAll)
            break
        
        experience = (s,r,a,new_s)
        memory.append(experience)
        if j==199:
            print("Reward %d after full episode"%(rAll))
            
        s = new_s
    batches=random.sample(memory,batch_size)
    states= np.array([batch[0] for batch in batches])
    rewards= np.array([batch[1] for batch in batches])
    actions= np.array([batch[2] for batch in batches])
    new_states= np.array([batch[3] for batch in batches])
    Qs =model.predict(states)
    new_Qs = model.predict(new_states)
    for i in range(len(rewards)):
        if rewards[i]==-100:
            Qs[i][0][actions[i]]=Qs[i][0][actions[i]]+ lr*(rewards[i]-Qs[i][0][actions[i]])
        else:
            Qs[i][0][actions[i]]= Qs[i][0][actions[i]]+ lr*(rewards[i]+gamma*np.max(new_Qs[i])-Qs[i][0][actions[i]])
    model.fit(states,Qs,verbose=0)
    epsilon=epsilon*epsilon_decay
env.close()

Episode 0, Failed! Reward 16
Episode 1, Failed! Reward 14
Episode 2, Failed! Reward 21
Episode 3, Failed! Reward 34
Episode 4, Failed! Reward 12
Episode 5, Failed! Reward 13
Episode 6, Failed! Reward 47
Episode 7, Failed! Reward 35
Episode 8, Failed! Reward 11
Episode 9, Failed! Reward 88
Episode 10, Failed! Reward 24
Episode 11, Failed! Reward 9
Episode 12, Failed! Reward 17
Episode 13, Failed! Reward 20
Episode 14, Failed! Reward 23
Episode 15, Failed! Reward 19
Episode 16, Failed! Reward 14
Episode 17, Failed! Reward 24
Episode 18, Failed! Reward 19
Episode 19, Failed! Reward 12
Episode 20, Failed! Reward 20
Episode 21, Failed! Reward 20
Episode 22, Failed! Reward 23
Episode 23, Failed! Reward 22
Episode 24, Failed! Reward 24
Episode 25, Failed! Reward 14
Episode 26, Failed! Reward 15
Episode 27, Failed! Reward 38
Episode 28, Failed! Reward 61
Episode 29, Failed! Reward 44
Episode 30, Failed! Reward 23
Episode 31, Failed! Reward 18
Episode 32, Failed! Reward 55
Episode 33, Failed! R

Episode 268, Failed! Reward 190
Episode 269, Failed! Reward 174
Episode 270, Failed! Reward 154
Episode 271, Failed! Reward 154
Episode 272, Failed! Reward 181
Episode 273, Failed! Reward 167
Episode 274, Failed! Reward 171
Episode 275, Failed! Reward 166
Episode 276, Failed! Reward 168
Episode 277, Failed! Reward 159
Episode 278, Failed! Reward 149
Episode 279, Failed! Reward 147
Episode 280, Failed! Reward 142
Episode 281, Failed! Reward 31
Episode 282, Failed! Reward 140
Episode 283, Failed! Reward 172
Episode 284, Failed! Reward 174
Episode 285, Failed! Reward 161
Episode 286, Failed! Reward 173
Episode 287, Failed! Reward 165
Episode 288, Failed! Reward 184
Episode 289, Passed! Reward 199
Episode 290, Failed! Reward 188
Episode 291, Failed! Reward 99
Episode 292, Failed! Reward 187
Episode 294, Failed! Reward 182
Episode 295, Passed! Reward 200
Episode 296, Failed! Reward 171
Episode 297, Failed! Reward 185
Episode 298, Failed! Reward 180
Episode 299, Failed! Reward 194
Episode 30

Episode 526, Failed! Reward 129
Episode 527, Failed! Reward 126
Episode 528, Failed! Reward 132
Episode 529, Failed! Reward 131
Episode 530, Failed! Reward 144
Episode 531, Failed! Reward 145
Episode 532, Failed! Reward 133
Episode 533, Failed! Reward 139
Episode 534, Failed! Reward 143
Episode 535, Failed! Reward 139
Episode 536, Failed! Reward 146
Episode 537, Failed! Reward 150
Episode 538, Failed! Reward 137
Episode 539, Failed! Reward 149
Episode 540, Failed! Reward 163
Episode 541, Failed! Reward 150
Episode 542, Failed! Reward 158
Episode 543, Failed! Reward 153
Episode 544, Failed! Reward 141
Episode 545, Failed! Reward 160
Episode 546, Failed! Reward 149
Episode 547, Failed! Reward 163
Episode 548, Failed! Reward 140
Episode 549, Failed! Reward 149
Episode 550, Failed! Reward 157
Episode 551, Failed! Reward 148
Episode 552, Failed! Reward 145
Episode 553, Failed! Reward 132
Episode 554, Failed! Reward 135
Episode 555, Failed! Reward 132
Episode 556, Failed! Reward 136
Episode 

Episode 783, Failed! Reward 126
Episode 784, Failed! Reward 154
Episode 785, Failed! Reward 183
Episode 786, Failed! Reward 125
Episode 787, Failed! Reward 118
Episode 788, Failed! Reward 143
Episode 789, Failed! Reward 113
Episode 790, Failed! Reward 121
Episode 791, Failed! Reward 136
Episode 792, Failed! Reward 140
Episode 793, Failed! Reward 157
Episode 794, Failed! Reward 136
Episode 795, Failed! Reward 126
Episode 796, Failed! Reward 134
Episode 797, Failed! Reward 149
Episode 798, Failed! Reward 164
Episode 799, Failed! Reward 176
Episode 800, Failed! Reward 168
Episode 801, Failed! Reward 156
Episode 802, Failed! Reward 163
Episode 803, Failed! Reward 162
Episode 804, Failed! Reward 172
Episode 805, Failed! Reward 171
Episode 806, Failed! Reward 154
Episode 807, Failed! Reward 161
Episode 808, Failed! Reward 149
Episode 809, Failed! Reward 155
Episode 810, Failed! Reward 137
Episode 811, Failed! Reward 153
Episode 812, Failed! Reward 169
Episode 813, Failed! Reward 147
Episode 

#### Plot of Episodes vs Reward

Deep Q Learning

![DQL](Reward_vs_Episode_DL_lr_0.500000_eps_1000.jpg)

Plot for Vanilla ```Q Learning```

![QLearning](Reward_vs_Episode_QL_lr_0.500000_eps_1000.jpg)

The above two pictures show that Deep Q Network reached the highest points faster than traditional Q Learning Algorithm

## Further Improvement

You can try and give some trials with different architectures, parameters and hyperparameters for improvement. 